In [17]:
import numpy as np
import torch
from torch import nn, optim
from torch.utils.data import DataLoader, TensorDataset

# Provided functions to generate and segment the time series
def generate_normalized_multivariate_time_series(n_features, total_length, amplitude=1.0):
    t = np.linspace(0, 100 * np.pi, total_length)
    series = np.zeros((total_length, n_features))
    for i in range(n_features):
        series[:, i] = amplitude * np.cos(t * (i + 1) / n_features) + 10
    
    series_sum = np.sum(series, axis=1, keepdims=True)
    series_normalized = series / series_sum
    
    return series_normalized

def segment_time_series(series, length):
    total_length, n_features = series.shape
    segments = []
    for start in range(0, total_length - length, length):
        segment = series[start:start + length]
        segments.append(segment)
    return np.stack(segments)

# Generating and segmenting the time series data
n_features = 4
length = 64
total_length = 1024

series = generate_normalized_multivariate_time_series(n_features, total_length)
series_x = series[:-1,]
series_y = series[1:,]

segments_x = segment_time_series(series_x, length)
segments_y = segment_time_series(series_y, length)

segments_tensor_x = torch.tensor(segments_x, dtype=torch.float)
segments_tensor_y = torch.tensor(segments_y, dtype=torch.float)

# Preparing inputs and targets
X = segments_tensor_x
Y = segments_tensor_y


In [18]:
class Seq2SeqModel(nn.Module):
    def __init__(self, n_features, n_heads, dim_feedforward, num_layers, dropout=0.1):
        super(Seq2SeqModel, self).__init__()
        self.encoder_layer = nn.TransformerEncoderLayer(d_model=n_features, nhead=n_heads, dim_feedforward=dim_feedforward, dropout=dropout)
        self.encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=num_layers)
        self.decoder_layer = nn.TransformerEncoderLayer(d_model=n_features, nhead=n_heads, dim_feedforward=dim_feedforward, dropout=dropout)
        self.decoder = nn.TransformerEncoder(self.decoder_layer, num_layers=num_layers)
        
    def forward(self, src):
        memory = self.encoder(src)
        output = self.decoder(memory)
        return output

# Model, loss, and optimizer
model = Seq2SeqModel(n_features=n_features, n_heads=2, dim_feedforward=2048, num_layers=2)
loss_function = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Training loop
def train(model, data_loader, optimizer, loss_function, epochs=1000):
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        for X_batch, Y_batch in data_loader:
            optimizer.zero_grad()
            output = model(X_batch)
            loss = loss_function(output, Y_batch)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
            
            if (epoch + 1) % 10 == 0:
                print(f'Epoch {epoch+1}, Loss: {total_loss/len(data_loader)}')

# Creating data loader
dataset = TensorDataset(X, Y)
data_loader = DataLoader(dataset, batch_size=32, shuffle=True)

# Training the model
train(model, data_loader, optimizer, loss_function)


Epoch 10, Loss: 0.8275810480117798
Epoch 20, Loss: 0.551434338092804
Epoch 30, Loss: 0.3408207893371582
Epoch 40, Loss: 0.1971943974494934
Epoch 50, Loss: 0.10844861716032028
Epoch 60, Loss: 0.05484980717301369
Epoch 70, Loss: 0.03263355419039726
Epoch 80, Loss: 0.022698648273944855
Epoch 90, Loss: 0.015056906268000603
Epoch 100, Loss: 0.011502319015562534
Epoch 110, Loss: 0.008033016696572304
Epoch 120, Loss: 0.004716662224382162
Epoch 130, Loss: 0.001869129715487361
Epoch 140, Loss: 0.0009665885590948164
Epoch 150, Loss: 0.0006454758695326746
Epoch 160, Loss: 0.0005785826360806823
Epoch 170, Loss: 0.00045588600914925337
Epoch 180, Loss: 0.00045683278585784137
Epoch 190, Loss: 0.0003878229181282222
Epoch 200, Loss: 0.0003819444100372493
Epoch 210, Loss: 0.0003949874662794173
Epoch 220, Loss: 0.00036775661283172667
Epoch 230, Loss: 0.00034041813341900706
Epoch 240, Loss: 0.0003270730667281896
Epoch 250, Loss: 0.0003269849403295666
Epoch 260, Loss: 0.0003230610745958984
Epoch 270, Loss:

In [19]:
outputs = model(X)

In [20]:
import plotly.graph_objects as go

def plot_model_output_vs_target(model_outputs, targets, batch_index=0, feature_index=0):
    # Extract the specified feature for the given batch from both the model outputs and targets
    model_output_series = model_outputs[batch_index, :, feature_index].detach().numpy()
    target_series = targets[batch_index, :, feature_index].numpy()
    
    # Create a range for the x-axis (timesteps)
    timesteps = list(range(model_output_series.shape[0]))
    
    # Create traces
    model_trace = go.Scatter(x=timesteps, y=model_output_series, mode='lines', name='Model Output')
    target_trace = go.Scatter(x=timesteps, y=target_series, mode='lines', name='Target')
    
    # Create the figure and add traces
    fig = go.Figure()
    fig.add_trace(model_trace)
    fig.add_trace(target_trace)
    
    # Add title and labels
    fig.update_layout(title=f'Model Output vs Target for Feature {feature_index}, Batch {batch_index}',
                      xaxis_title='Timestep',
                      yaxis_title='Value')
    
    # Show the figure
    fig.show()

# Assuming `y` and `Y` are your model outputs and targets, respectively
# Adjust batch_index and feature_index as needed
plot_model_output_vs_target(outputs, Y, batch_index=2, feature_index=3)